In [12]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from gurobipy import *
from scipy.spatial.distance import cdist, euclidean
from scipy.stats import truncnorm
from scipy.sparse import csr_matrix
from time import time
import networkx as nx

#path = "C:/Users/a.rojasa55/OneDrive - Universidad de los Andes/Documentos/MOPTA-23/Data/"
path = "C:/Users/ari_r/OneDrive - Universidad de los Andes/Documentos/MOPTA-23/Data/"
vehicles = pd.read_csv(path+'MOPTA2023_car_locations.csv', sep = ',', header = None)

stations = pd.read_csv(path+"fuel_stations.csv")

northern = (-79.761960, 42.269385)
southern = (-76.9909,39.7198)
western = (-80.519400, 40.639400)
eastern = (-74.689603, 41.363559)

stations_loc = stations[["Longitude","Latitude"]]
stations_loc["Latitude"] = (stations["Latitude"]-southern[1])*69*165/178
stations_loc["Longitude"] = (stations["Longitude"]-western[0])*53

stations = stations_loc[(stations_loc["Longitude"] <= 290) & (stations_loc["Latitude"] <= 150)]
stations.rename(columns={"Longitude": 0, "Latitude":1}, inplace=True)

C:\Users\ari_r\AppData\Local\Temp\ipykernel_14148\3189103986.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_loc["Latitude"] = (stations["Latitude"]-southern[1])*69*165/178
C:\Users\ari_r\AppData\Local\Temp\ipykernel_14148\3189103986.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_loc["Longitude"] = (stations["Longitude"]-western[0])*53
C:\Users\ari_r\AppData\Local\Temp\ipykernel_14148\3189103986.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

# Instance Generation

In [13]:
def feasibility_vehic_to_locations(df_vehic,v,r, df_loc, steps_per_hour, driving_speed, charging_speed, T):
    distances = {s:euclidean((df_vehic.loc[v,0], df_vehic.loc[v,1]), (df_loc.loc[s,0], df_loc.loc[s,1])) for s in range(1,df_loc.shape[0]+1)}
    p = {s:250-(r-distances[s]) for s in range(1,df_loc.shape[0]+1)}
    a = {s:int(distances[s]*steps_per_hour/driving_speed)+1 for s in range(1,df_loc.shape[0]+1)}
    k = {s:int(p[s]*steps_per_hour/charging_speed)+1 for s in range(1,df_loc.shape[0]+1)}
    return sum([1 if r/distances[s] >= 1 and a[s]+k[s]-1<=T[-1] else 0 for s in range(1,df_loc.shape[0]+1)])


def distance_matrix(df1, df2):
    # Extract the x and y coordinates as arrays
    x1 = np.array(df1[0])
    y1 = np.array(df1[1])
    x2 = np.array(df2[0])
    y2 = np.array(df2[1])
    
    # Compute the pairwise distances between the two sets of points
    distances = cdist(np.column_stack((x1, y1)), np.column_stack((x2, y2)))
    
    # Convert the distance matrix to a dictionary
    distance_dict = {}
    for i in range(1,len(df1)+1):
        for j in range(1,len(df2)+1):
            location1 = i
            location2 = j
            distance = distances[i-1,j-1]
            distance_dict[(location1, location2)] = round(distance,2)
            
    return distance_dict

In [16]:
''' Number of vehicles and stations '''

C = list(range(1,201))
S = list(range(1,21))

''' Fixed parameters '''

driving_speed = 75 # miles per hour
steps_per_hour = 6 # each time step is ten minutes
charging_speed = 75*12/7 # miles per hour
T = list(range(1,int(steps_per_hour*2.5)+1)) # Planning horizon

loc, scale, min_v, max_v= 100, 50, 20, 250
a, b = (min_v - loc) / scale, (max_v - loc) / scale

''' Feasible instance generator '''

d = distance_matrix(vehicles,stations)

random.seed(453)
''' Feasible instance creation '''
feas_long = 145; feas_lat = 75

stations = stations[(stations[0] >= feas_long) & (stations[1] <= feas_lat)]
stations.index = np.arange(1, len(stations) + 1)
indices = random.sample(range(len(stations)), len(S))
stations = stations.iloc[indices]
stations.index = np.arange(1, len(stations) + 1)

vehicles = vehicles[(vehicles[0] >= feas_long) & (vehicles[1] <= feas_lat)]
vehicles.index = np.arange(1, len(vehicles) + 1)

feasible = 0; sample_vehic = pd.DataFrame({0:[], 1:[], "range":[]})
while feasible < len(C):
    index = random.randint(1,vehicles.shape[0]-1)
    range_real = truncnorm.rvs(a = a, b = b, loc = loc, scale = scale, size = 1)

    if feasibility_vehic_to_locations(vehicles, index, range_real, stations, steps_per_hour, driving_speed, charging_speed, T) >= len(S):
        feasible += 1
        sample_vehic = pd.concat([sample_vehic,pd.DataFrame({0:vehicles.loc[index,0], 1:vehicles.loc[index,1], "range":[range_real]},index=[0])])
        sample_vehic.index = np.arange(1, len(sample_vehic) + 1)

d = distance_matrix(sample_vehic,stations)
r = {c:sample_vehic.loc[c,"range"][0] for c in C}
p = {(c,s):250-(r[c]-d[c,s]) for c in C for s in S}
a = {(c,s):int(d[c,s]*steps_per_hour/driving_speed)+1 for c in C for s in S}
k = {(c,s):int(p[c,s]*steps_per_hour/charging_speed)+1 for c in C for s in S}
y = {s:6 for s in S}
cf = 5500
cd = 0.041
cw = 0.0388

# SPPRC

In [17]:
def get_graph(s,K,cd,cw,a,d,p,pi,sigma):

    vehic_rc = {k:cd*d[k,s]+cw*p[k,s]-pi[f"V{k}"] for k in K}
    Ks = [k for k in K if vehic_rc[k]<0]

    V = ["s"] + Ks + ["e"]
    A = [(i,j) for i in V for j in V if i!=j and i!="e" and j!="s" and a[i,s] < a[j,s] and (i,j)!=("s","e")]

    rc = {arc:vehic_rc[arc[1]]-sigma if arc[0]=="s" else (0 if arc[1]=="e" else vehic_rc[arc[1]]) for arc in A}
    
    return V,A,rc

def get_dummy_graph(s,K,cd,cw,a,d,p,pi,sigma):

    vehic_rc = {k:cd*d[k,s]+cw*p[k,s]-pi[f"V{k}"] for k in K}

    V = ["s"] + K + ["e"]
    A = [(i,j) for i in V for j in V if i!=j and i!="e" and j!="s" and a[i,s] < a[j,s] and (i,j)!=("s","e")]

    rc = {arc:vehic_rc[arc[1]]-sigma if arc[0]=="s" else (0 if arc[1]=="e" else vehic_rc[arc[1]]) for arc in A}
    
    return V,A,rc

def vertices_extensions(V,A):
    
    G = nx.DiGraph()
    G.add_nodes_from(V); G.add_edges_from(A)
    outbound_arcs = {}
    for v in V:
        outbound_arcs[v] = list(G.out_edges(v))
    
    return outbound_arcs

def label_algorithm(s,V,K,T,r,t,a,ext,pi,sigma):

    def label_extension(l,arc,dominated):
        for m in range(1):
            i = arc[0]; j = arc[1]
            new_label = [[], 0, 0, 0]

            ''' Check waiting line feasibility '''
            if a[j,s] < l[2]: break
            if a[j,s] > l[3]: new_label[2] = l[2]
            else: new_label[2] = l[3]

            ''' Check time consumption feasibility '''
            new_label[3] = l[3] + max(0,a[j,s]-l[3]) + t[j,s]
            if new_label[3] > T: break
            
            ''' Update the resources consumption '''
            new_label[0] += l[0] + [j]
            new_label[1] = l[1] + r[(i,j)]
            
            if j == "e":
                done.append(new_label)
            else:
                new_labels.append(new_label)
                label_dominance(new_label,dominated)
    
    '''def label_dominance(list1, list2=None):
            if list2 is None: labels_copy = list1.copy()
            else: labels_copy = list2

            for l1 in labels_copy:
                for l in list1:
                    if l1 != l:
                        if set(l1[0]).issubset(l[0]):
                            #print(f"Route {l[0]}  dominates {l1[0]}")
                            list1.remove(l1); break'''
    
    def label_dominance(new_label,dominated):
        for l in range(len(labels)):
            if set(labels[l][0]).issubset(set(new_label[0])):
                dominated[l] = True

    ''' Labels list '''
    # Index: number of label
    # 0: route
    # 1: cumulative reduced cost
    # 2: last moment where there was a vehicle waiting in line to use the charger
    # 3: cumulative time consumption
    labels = [ [[arc[0], arc[1]], r[arc], a[arc[1],s], a[arc[1],s]+t[arc[1],s]] for arc in ext["s"]]
    done = []

    while len(labels) > 0:
        
        L = range(len(labels))
        new_labels = []
        dominated = {l:False for l in L}
        for l in L:
            for arc in ext[labels[l][0][-1]]:
                if not dominated[l]:
                    label_extension(labels[l], arc, dominated)

        del labels[:len(L)]
        labels = new_labels.copy()

        ''' Dominance of done labels over new labels '''
        #label_dominance(labels,done)

        ''' Dominance among new labels '''
        #label_dominance(labels)
    
    ''' Dominance among done labels '''
    #label_dominance(done)

    routes = []
    for l in range(len(done)):
        # If reduced cost is negative
        if done[l][1] < -0.001:
            col = {k:1 if k in done[l][0] else 0 for k in K}
            routes.append((col,done[l][1]+sum(pi[f"V{k}"]*col[k] for k in K)+sigma))

    return routes

In [18]:
def chorizo_maker_new(S,K,K_s,cd,cw,d,p):

    routes = {0:[]}
    for k in K:
        routes[0].append(({kk:1 if kk == k else 0 for kk in K},250*(cd+cw)))

    np.random.seed(0)
    for s in S:
        rand = np.random.choice(K_s[s])
        routes[s] = [({k:1 if k == rand else 0 for k in K_s[s]},cd*d[rand,s]+cw*p[rand,s])]

    return routes

def second_stage_ESPP(S,K,T,y,a,d,r,p,t,cd,cw):

    def feas(v,s):
        return d[v,s]<=r[v] and a[v,s]+t[v,s]<=T
    
    i = 0

    S_k = {k:[s for s in S if feas(k,s)] for k in K}
    K_s = {s:[k for k in K if feas(k,s)] for s in S}
    routes = chorizo_maker_new(S,K,K_s,cd,cw,d,p)

    objs = []
    time0 = time()

    while True:
        i += 1

        pi, infeasible, objMP,zz = master_problem(S,K,y,routes,S_k,K_s,output=0)
        print(f"Iteration {i}:\n{len(infeasible)} vehicles\tMP obj: {round(objMP,2)}\ttime: {round(time()-time0,2)}s")

        opt = {}
        for s in S:
            V,A,rc = get_graph(s,K_s[s],cd,cw,a,d,p,pi,pi[f"S{s}"])
            ext = vertices_extensions(V,A)
            opt[s] = label_algorithm(s,V,K,T,rc,t,a,ext,pi,pi[f"S{s}"])
            print(f"Station {s}: {len(opt[s])} new columns")
            routes[s] += opt[s]

        if sum(len(opt[s]) for s in S) == 0: break
    
    if sum([1 for s in S for k in K_s[s] if zz[k,s]-int(zz[k,s]) > 0]) > 0:
        pi, infeasible, objMP, zz = master_problem(S,K,y,routes,S_k,K_s,output=0,integer=1)

    return infeasible, objMP, zz, objs

In [19]:
def plot_graph(s,K,V,A,a,t,Route=None,color_arcs=False,feas=True):

    fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(16,len(V)-2))

    if color_arcs: col="#D60093"
    else: col="darkgray"
    for arc in A:
        if arc[0] == "s": ax1.plot([a[arc[0],s],a[arc[1],s]],[(V[-2]+V[1])/2,arc[1]],linestyle="-",color=col,marker=None)
        elif arc[1] == "e": ax1.plot([a[arc[0],s],a[arc[1],s]],[arc[0],(V[-2]+V[1])/2],linestyle="-",color=col,marker=None)
        else: ax1.plot([a[arc[0],s],a[arc[1],s]],[arc[0],arc[1]],linestyle="-",color=col,marker=None)

    ''' Charging route '''
    if Route is not None:
        if feas: col = "limegreen"
        else: col = "firebrick"
        ax1.plot([a["s",s],a[Route[0],s]],[(V[-2]+V[1])/2,Route[0]],linestyle="-",color=col,marker=None)
        for i in range(len(Route)-1):
            ax1.plot([a[Route[i],s],a[Route[i+1],s]],[Route[i],Route[i+1]],linestyle="-",color=col,marker=None)
        ax1.plot([a[Route[-1],s],a["e",s]],[Route[-1],(V[-2]+V[1])/2],linestyle="-",color=col,marker=None)
    
    ''' Vehicle nodes '''
    for v in V:
        if v in ["s","e"]: ax1.plot(a[v,s],(V[-2]+V[1])/2,marker="o",markersize=20,color="maroon")
        else: ax1.plot(a[v,s],v,marker="o",markersize=24,color="navy"); ax1.text(x=a[v,s],y=v,s=f"k{v}",color="white",va="center",ha="center",fontsize=14,fontname="Cambria")

    ax1.set_xlim(-0.05,T+0.05); ax1.set_xlabel("Time (minutes)",fontsize=14,fontname="Century Gothic")
    ax1.set_ylim(0.5,len(V)-2+0.5); #ax1.set_ylabel("Vehicles",fontsize=14)

    timesteps = [t*T/10 for t in range(11)]
    ax1.set_xticks(timesteps); ax1.set_xticklabels([int(t*60) for t in timesteps],fontsize=14)

    #ax1.set_yticks(K); ax1.set_yticklabels([f"k{k}" for k in K], fontsize=14)
    ax1.set_yticks([])
    ax1.spines[["top","left","right"]].set_visible(False)

    for tick in ax1.get_xticklabels():
        tick.set_fontname("Cambria")

    ax1.invert_yaxis()
    ax1.grid(which="minor",axis="x")
    ax1.set_title("Charging route graph representation",fontsize=14,fontname="Century Gothic")

    cum_t = 0; i = 1
    for k in Route:
        if cum_t > a[k,s]: ax2.barh(y=i,width=cum_t-a[k,s],height=1,left=a[k,s],color="turquoise")
        ax2.barh(y=i,width=t[k,s],height=1,left=cum_t+max(0,a[k,s]-cum_t),color="forestgreen")
        cum_t += max(0,a[k,s]-cum_t) + t[k,s]; i+=1

    ax2.set_xlim(0,T)
    ax2.set_ylim(0.5,i-0.5)

    timesteps = [t*T/10 for t in range(11)]
    ax2.set_xticks(timesteps); ax2.set_xticklabels([int(t*60) for t in timesteps],fontsize=14)
    ax2.set_yticks([k for k in range(1,i)]); ax2.set_yticklabels([f"k{k}" for k in Route],fontsize=14)

    ax2.bar(x=0,height=0,color="turquoise",label="Waiting"); ax2.bar(x=0,height=0,color="forestgreen",label="Charging")
    leg = ax2.legend(loc="lower left",ncol=2)

    for text in leg.get_texts():
        text.set_fontfamily("Century Gothic")

    ax2.set_xlabel("Time (minutes)",fontsize=14,fontname="Century Gothic")
    ax2.set_ylabel("Vehicles",fontsize=14,fontname="Century Gothic")
    ax2.set_title("Charging route schedule",fontsize=14,fontname="Century Gothic")

    for tick in ax2.get_xticklabels():
        tick.set_fontname("Cambria")
    for tick in ax2.get_yticklabels():
        tick.set_fontname("Cambria")

    ax2.invert_yaxis()

In [24]:
cd = 0.041
cw = 0.0388

K = list(range(1,5))
T = 100/60
a = {("s",1):0, (1,1):10/60, (2,1):30/60, (3,1):40/60, (4,1):80/60, ("e",1):T}
t = {("s",1):0, (1,1):40/60, (2,1):30/60, (3,1):30/60, (4,1):20/60, ("e",1):0}
V,A,rc = get_dummy_graph(1,K,cd,cw,a,d,p,{f"V{k}":10 for k in K},0)
ext = vertices_extensions(V,A)
routes = label_algorithm(1,V,K,T,rc,t,a,ext,{f"V{k}":10 for k in K},0)


In [25]:
routes

[({1: 0, 2: 0, 3: 1, 4: 1}, 18.400463619413237),
 ({1: 1, 2: 1, 3: 0, 4: 1}, 29.945237205845885)]

In [20]:
K = C
T = 2.5
t = {(k,s):p[k,s]/charging_speed for k in K for s in S}
a = {(k,s):d[k,s]/driving_speed for k in K for s in S}
a.update({("s",s):0 for s in S})
a.update({("e",s):T for s in S})
t.update({("e",s):0 for s in S})

In [21]:
infeasibleSPP2, objMP2, zz2, obj2 = second_stage_ESPP(S,K,T,y,a,d,r,p,t,cd,cw)

Iteration 1:
183 vehicles	MP obj: 3758.44	time: 0.46s
Station 1: 129 new columns
Station 2: 190 new columns
Station 3: 182 new columns
Station 4: 181 new columns
Station 5: 182 new columns
Station 6: 182 new columns
Station 7: 187 new columns
Station 8: 190 new columns
Station 9: 188 new columns
Station 10: 189 new columns
Station 11: 184 new columns
Station 12: 185 new columns
Station 13: 189 new columns
Station 14: 186 new columns
Station 15: 178 new columns
Station 16: 189 new columns
Station 17: 192 new columns
Station 18: 187 new columns
Station 19: 192 new columns
Station 20: 189 new columns
Iteration 2:
56 vehicles	MP obj: 2133.07	time: 9.62s
Station 1: 15 new columns
Station 2: 173 new columns
Station 3: 54 new columns
Station 4: 125 new columns
Station 5: 164 new columns
Station 6: 109 new columns
Station 7: 171 new columns
Station 8: 130 new columns
Station 9: 123 new columns
Station 10: 123 new columns
Station 11: 112 new columns
Station 12: 167 new columns
Station 13: 127 n

In [23]:
250*4500*(0.041+0.0388)/5500

16.322727272727274

225.0